<a href="https://colab.research.google.com/github/svechino/TravelTide/blob/main/KMeans%20updatec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and data

In [ ]:
pip install plotly

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import plotly.express as px
from plotly import graph_objects as go

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading users indexes:
perks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_perks.csv')
print(perks.shape)

# Loading information about users:
users_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/users_data.csv')
print(users_data.shape)

(5998, 7)
(5998, 11)


In [ ]:
# Filling NaN values with zero:
perks.fillna(0, inplace=True)
perks.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,23557,0.00268,0.00,0.0,0.000000,0.0,0.333525
1,94883,0.00000,0.10,0.0,0.000000,1.0,0.418203
2,101486,0.00000,0.00,0.0,0.000000,0.5,0.549539
3,101961,0.00000,0.08,0.0,0.000058,1.0,0.176267
4,106907,0.00000,1.00,0.5,0.000000,1.0,0.434310


In [ ]:
users_data.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date
0,94883,1972-03-16,F,True,False,usa,kansas city,MCI,39.297,-94.714,2022-02-07
1,101486,1972-12-07,F,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17
2,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17
3,106907,1978-11-17,F,True,True,usa,miami,TNT,25.862,-80.897,2022-02-24
4,118043,1972-05-04,F,False,True,usa,los angeles,LAX,33.942,-118.408,2022-03-10


In [ ]:
# Checking format data:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           5998 non-null   int64  
 1   birthdate         5998 non-null   object 
 2   gender            5998 non-null   object 
 3   married           5998 non-null   bool   
 4   has_children      5998 non-null   bool   
 5   home_country      5998 non-null   object 
 6   home_city         5998 non-null   object 
 7   home_airport      5998 non-null   object 
 8   home_airport_lat  5998 non-null   float64
 9   home_airport_lon  5998 non-null   float64
 10  sign_up_date      5998 non-null   object 
dtypes: bool(2), float64(2), int64(1), object(6)
memory usage: 433.6+ KB


We have identified a few issues in our dataset that require attention:

1. Convert 'Birthdate' to date format.
2. Calculate the age based on 'Birthdate'.
3. Check the 'Gender' column for the number of unique values and consider converting this data to boolean.


In [ ]:
# Convert Birthdate to date format:
users_data['birthdate'] = pd.to_datetime(users_data['birthdate'], errors='coerce')

In [ ]:
# Checking unique values in "Gender" column:
users_data['gender'].value_counts()

F    5292
M     695
O      11
Name: gender, dtype: int64

 Since amount of O-values is insignificant (only 0.18% of total) we will replace them by most frequent
 value ("F") and convert this column to boolean

In [ ]:
# Calculation users age:
users_data['age'] = (pd.Timestamp.now() - users_data['birthdate'])//np.timedelta64(1,'Y')

# Replacing O-values by "F" in "Gender":
replace_values = {'F' : True, 'O' : True, 'M' : False }
users_data['gender'] = users_data['gender'].map(replace_values)

In [ ]:
users_data.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,age
0,94883,1972-03-16,True,True,False,usa,kansas city,MCI,39.297,-94.714,2022-02-07,51
1,101486,1972-12-07,True,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17,50
2,101961,1980-09-14,True,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17,42
3,106907,1978-11-17,True,True,True,usa,miami,TNT,25.862,-80.897,2022-02-24,44
4,118043,1972-05-04,True,False,True,usa,los angeles,LAX,33.942,-118.408,2022-03-10,51


In [ ]:
# We will drop unnecessary columns from the 'users_data' DataFrame to keep only the relevant information:
users_data.drop(['birthdate','home_country',
                 'home_city', 'home_airport',
                 'home_airport_lat', 'home_airport_lon',
                 'sign_up_date'], axis='columns', inplace=True)

In [ ]:
# We will merge two datasets in one base on the user ID number:
data = users_data.merge(perks, how='inner')
data.head()

,user_id,gender,married,has_children,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,94883,True,True,False,51,0.000000,0.10,0.0,0.000000,1.0,0.418203
1,101486,True,True,True,50,0.000000,0.00,0.0,0.000000,0.5,0.549539
2,101961,True,True,False,42,0.000000,0.08,0.0,0.000058,1.0,0.176267
3,106907,True,True,True,44,0.000000,1.00,0.5,0.000000,1.0,0.434310
4,118043,True,False,True,51,0.000725,0.20,0.0,0.000000,0.4,0.457949


In [ ]:
data.loc[data.isnull().any(axis=1)]

,user_id,gender,married,has_children,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index


In [ ]:

def show_clusters_on_plot(df, x_name, y_name, cluster_name):
    plt.figure(figsize = (10,10))
    sns.scatterplot(df[x_name], df[y_name],
          hue = df[cluster_name], palette = 'Paired'
       )
    plt.title('{} vs {}'.format(x_name, y_name))
    plt.show()

In [ ]:
# standardize the data
sc = StandardScaler()
x_sc = sc.fit_transform(data)

In [ ]:
km = KMeans(n_clusters=5, random_state=0)
# predict the clusters for observations (the algorithm assigns them a number from 0 to 2)
labels = km.fit_predict(x_sc)

# store cluster labels in the field of our dataset
data['cluster'] = labels
data.sample(10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,gender,married,has_children,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,cluster
4704,549608,True,True,True,58,0.000000,0.000000,0.0,0.000000,0.000000,0.464862,2
5725,623087,True,False,False,47,0.000000,0.000000,0.0,0.000000,1.000000,0.483871,4
1443,537904,True,False,True,32,0.000000,0.200000,0.0,0.000789,0.800000,0.336982,4
3815,519430,True,False,False,34,0.000398,0.100000,0.0,0.000000,1.000000,0.416475,4
5561,601582,False,False,True,52,0.000000,0.000000,0.0,0.000000,0.500000,0.380184,0
5244,575253,False,True,True,43,0.000000,0.133333,0.0,0.000265,0.666667,0.190092,0
4561,544391,False,True,True,43,0.000740,0.100000,0.0,0.001625,0.666667,0.423387,0
3028,374340,True,True,False,34,0.000000,0.200000,0.0,0.000000,0.000000,0.483871,2
5294,578488,True,False,False,51,0.000000,0.200000,0.0,0.011881,1.000000,0.413018,4
1199,529734,True,True,False,38,0.000045,0.200000,0.0,0.000075,0.800000,0.350806,2


In [ ]:
mean_by_clusters = data.groupby(['cluster']).mean().reset_index()
mean_by_clusters[['cluster', 'age',
                       'hotel_hunter_index',
                       'average_bags_scaled',
                       'cancellation_rate_scaled',
                       'bargain_hunter_index',
                        'combined_booking_scaled','session_intencity_index']]

,cluster,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,0,40.804545,0.000569,0.102493,0.016022,0.001760,0.679766,0.400789
1,1,39.199584,0.000657,0.149751,0.318676,0.003620,0.758957,0.376595
2,2,47.944841,0.000670,0.107059,0.002746,0.001851,0.686425,0.413835
3,3,28.233146,0.000344,0.017228,0.000201,0.000956,0.079839,0.411567
4,4,38.729677,0.000636,0.117785,0.002298,0.001748,0.832127,0.411567


In [ ]:
median_by_clusters = data.groupby(['cluster']).median().reset_index()
median_by_clusters[['cluster', 'age',
                       'hotel_hunter_index',
                       'average_bags_scaled',
                       'cancellation_rate_scaled',
                       'bargain_hunter_index',
                        'combined_booking_scaled','session_intencity_index']]

,cluster,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,0,42.0,0.0,0.100000,0.000000,0.0,0.75,0.400922
1,1,40.0,0.0,0.133333,0.333333,0.0,0.75,0.373708
2,2,47.0,0.0,0.100000,0.000000,0.0,0.75,0.411290
3,3,25.0,0.0,0.000000,0.000000,0.0,0.00,0.411290
4,4,39.0,0.0,0.100000,0.000000,0.0,1.00,0.411290


In [ ]:
numeric_vars = data[['cluster','age',
                       'hotel_hunter_index',
                       'average_bags_scaled',
                       'cancellation_rate_scaled',
                       'bargain_hunter_index',
                        'combined_booking_scaled','session_intencity_index']].copy()

In [ ]:
categorical_vars = data[['gender', 'married', 'has_children', 'cluster']].copy()

In [ ]:
for i in numeric_vars.columns:
    fig = px.box(x = numeric_vars['cluster'], y = numeric_vars[i])
    fig.update_layout(title = i, xaxis_title = 'Cluster number', yaxis_title = i)
    fig.show()

In [ ]:
for i in numeric_vars.columns:
    fig = px.histogram(numeric_vars, x=i, color='cluster', title=i, barmode='overlay')
    fig.update_layout(yaxis_title='Frequency', xaxis_title=i)
    fig.show()

In [ ]:
for i in categorical_vars:
    fig = px.histogram(categorical_vars, x='cluster', color=i, title=i, barmode='overlay')
    fig.update_layout(yaxis_title='Frequency', xaxis_title=i)
    fig.show()

### Demographic characterictis:
**Age:** The youngest group among our customers is in cluster 3, with a median age of 25, followed by cluster 4, where the median age is 39 years. Cluster 2 comprises the oldest customers, with a median age of 47 years

**Gender:** Cluster 0 consists exclusively of male customers, and there is an insignificant number of male users in cluster 1. All other clusters are comprised entirely of female customers

**Marriage:** Cluster 2 is predominantly composed of married customers, while cluster 4 consists entirely of unmarried customers. All other clusters exhibit a mix of marital statuses in varying proportions

**Parental status:** All clusters show a similar distribution of users with and without children, with a higher proportion of users without children across all clusters. Cluster 4 has the highest share of users without children, accounting for 76% of total users in the cluster, followed by clusters 0 and 1, both at 66%


### Indexes by Clusters:

**Cluster 0:** This cluster does not have a leading position in any of the indexes. They perform well in Cancellation Rate (2nd place) and Bargain Hunter Index (3rd place), indicating an interest in perks such as 'No Cancellation Fees' or 'Exclusive Discounts.'

**Cluster 1:** Cluster 1 holds leading positions in multiple indexes, including Average Bags, Cancellation Rate, and Bargain Hunter Index. This suggests their interest lies in both flight discounts and accompanying services.

**Cluster 2:** Cluster 2 excels in the Hotel Hunter Index and ranks 2nd in Bargain Hunter Index, indicating a strong interest in hotel and flight discounts.

**Cluster 3:** This cluster ranks last in most indexes but holds the leading position in Session Activity, showing a keen interest in hunting for discounts.

**Cluster 4:** Cluster 5 leads in the Combined Booking Index, indicating a preference for perks related to combined bookings, such as '1 Free Night Hotel with Flight.'

In summary, we can label our clusters based on their interests in the following perks:

Cluster 0: 'No Cancellation Fees'
Cluster 1: 'Free Checked Bag'
Cluster 2: 'Free Hotel Meal'
Cluster 3: 'Exclusive Discount'
Cluster 4: '1 Free Night Hotel with Fligh'